In [1]:
import pygad
import numpy as np
import pandas as pd
from CODECbreakCode.AudioMixer import FullTrackAudioMixer
import CODECbreakCode.Evaluator as Evaluator
from CODECbreakCode.Evaluator import MeasureHAAQIOutput
import argparse
from Optimiser.config import get_config, normalize_action, denormalize_action

In [2]:
import warnings
warnings.filterwarnings("ignore", "Possible clipped samples in output.")
warnings.filterwarnings("ignore",message="Warning: input samples dtype is np.float64. Converting to np.float32")

In [3]:
Reggae_Mixing_Path = '/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae'
Reggae_Noise_Generator_MP3 = FullTrackAudioMixer(Reggae_Mixing_Path)
#Noise_Generator_MP3.ManipulateInitGAIN([0, 0, 0, 0])
Reggae_Referece_File = Reggae_Noise_Generator_MP3.TestDynCompTrack([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"Reference_IN_O.wav",isNormalised=False,isCompensated=True)
print(f"Referece_File:{Reggae_Referece_File}")

Reggae_Referece_MP3File = Evaluator.Mp3LameLossyCompress(Reggae_Referece_File,64)
print(f"Referece_MP3File:{Reggae_Referece_MP3File}")

Vocal duration orginal is 17.462666666666667 seconds, now is the 8.0, the audio changing to the MONO
Drum duration orginal is 17.462666666666667 seconds, now is the 8.0, the audio changing to the MONO
Bass duration orginal is 17.462666666666667 seconds, now is the 8.0, the audio changing to the MONO
Other duration orginal is 17.462666666666667 seconds, now is the 8.0,  the audio changing to the MONO
Mixing File Load Sucessful
Referece_File:/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result/Reference_IN_O.wav
Referece_MP3File:/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result_Mp3_Wav/Reference_IN_O_64kbps.wav


In [4]:
Reggae_MeasureHAAQI = MeasureHAAQIOutput(Reggae_Referece_MP3File)#Initilize the HAAQI with a permanent reference
Reggae_MeasureHAAQI.MeasureHAQQIOutput(Reggae_Referece_MP3File) #Test on how far from itself to itsel

0.9952523970516826

In [5]:
import tempfile
import os
BASE_DIR = "/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result"
TMP_SUBDIR = os.path.join(BASE_DIR, "tmp")

# make sure it exists once at startup:
os.makedirs(TMP_SUBDIR, exist_ok=True)
def haaqi_reward_muti_fn(solution: np.ndarray, is_normalised=True) -> float:
    if is_normalised:
        solution = denormalize_action(solution)
#    print(f'solution:{solution}')
 
    # Create a unique temp‐file name
    fd, degradated_filename = tempfile.mkstemp(prefix="dynC_", suffix=".wav")
    os.close(fd)  # we’ll let your compressor write to that path

    try:
        gener_Audio = Reggae_Noise_Generator_MP3.TestDynCompTrack(solution,degradated_filename,isNormalised=False,isCompensated=True)
        gener_Audio_mp3 = Evaluator.Mp3LameLossyCompress(gener_Audio, 64)
        score = Reggae_MeasureHAAQI.MeasureHAQQIOutput(gener_Audio_mp3)
    finally:
        # clean up
        try:
            os.remove(degradated_filename)
        except OSError:
            pass

    return round(1 - score, 4)

In [6]:
import Optimiser.continous_RL_train_PPO
from Optimiser.continous_RL_train_PPO import continous_RL_train_PPO as CRLTrain

2025-07-26 16:28:12.160194: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-26 16:28:12.210297: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-26 16:28:12.210340: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-26 16:28:12.215020: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-26 16:28:12.233928: I tensorflow/core/platform/cpu_feature_guar

In [7]:
trainner = CRLTrain(sub_episode_length=30, sub_episode_num_single_batch=2, env_num=2)
trainner.set_environments(haaqi_reward_muti_fn)
trainner.train(update_num=10, eval_intv=5)

number of sub_episodes used for a single param update: 4
train_env.batch_size = parallel environment number =  2
update_num: 10, eval_intv: 5
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=True)` instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=True)` instead.


Epoch 0: policy_loss=-0.042  value_loss=1807.923  entropy_loss=-0.021  clip_fraction=0.058
final reward before udpate: -1000000000.0
final reward after udpate: 0.2197
updated final_solution= [ 0.0000e+00  2.5000e+00  5.5200e+00  5.0000e+02 -2.2000e-01  1.0000e+00
  2.0000e+01  2.7043e+02 -2.7700e+00  1.0000e+01  2.0000e+01  1.0000e+02
 -2.5440e+01  1.0000e+01  1.0000e+00  1.0000e+02]
train_step no.= 0
best_solution of this generation= [ 0.0000e+00  2.5000e+00  5.5200e+00  5.0000e+02 -2.2000e-01  1.0000e+00
  2.0000e+01  2.7043e+02 -2.7700e+00  1.0000e+01  2.0000e+01  1.0000e+02
 -2.5440e+01  1.0000e+01  1.0000e+00  1.0000e+02]
best step reward= 0.2197
avg step reward= 0.11347844
act_std: [-3.9000e-01  9.8800e+00  1.9750e+01  4.9482e+02 -3.9000e-01  9.8800e+00
  1.9750e+01  4.9482e+02 -3.9000e-01  9.8800e+00  1.9750e+01  4.9482e+02
 -3.9000e-01  9.8800e+00  1.9750e+01  4.9482e+02]
act_mean: [-15.    5.5  10.5 300.  -15.    5.5  10.5 300.  -15.    5.5  10.5 300.
 -15.    5.5  10.5 300. ]

In [ ]:
Reggae_Noise_Generator_MP3.EraseTheMp3Mixing()